In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

import nest_asyncio
nest_asyncio.apply()

MISTRAL_API_KEY = os.environ["MISTRAL_API_KEY"]

In [ ]:
from llama_index.llms.mistralai import MistralAI
from llama_index.core import Settings

# llm = AzureOpenAI(
#     api_key=os.environ["AZURE_OPENAI_API_KEY"],
#     azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
#     azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
#     api_version=os.environ["AZURE_OPENAI_API_VERSION"],
#     temperature=0.0,
# )

llm = MistralAI(temperature=0.0, model='mistral-small-latest')
Settings.llm = llm

# embed_model = MistralAIEmbedding()

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5"
)

Settings.embed_model = embed_model

In [3]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore, Neo4jGraphStore

URI = os.environ["NEO4J_URI"]
USERNAME = os.environ["NEO4J_USERNAME"]
PASSWORD = os.environ["NEO4J_PASSWORD"]

graph_store = Neo4jPropertyGraphStore(
    username=USERNAME,
    password=PASSWORD,
    url=URI,
)

# graph_store = Neo4jGraphStore(
#     username=USERNAME,
#     password=PASSWORD,
#     url=URI,
# )

In [12]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader('md').load_data()

In [4]:
from llama_index.core import PropertyGraphIndex, KnowledgeGraphIndex
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor, DynamicLLMPathExtractor, SimpleLLMPathExtractor

# Extract graph from documents
# index = PropertyGraphIndex.from_documents(
#     documents,
#     embed_model=embed_model,
#     kg_extractors=[
#         # SchemaLLMPathExtractor(
#         #     llm=llm,
#         #     # possible_entities=ENTITIES,
#         #     # possible_relations=RELATIONS,
#         #     # kg_validation_schema=VALIDATION_SCHEMA,
#         # )
#         # DynamicLLMPathExtractor(
#         #     llm=llm,
#         # )
#         SimpleLLMPathExtractor(
#             llm=llm,
#         )
#     ],
#     property_graph_store=graph_store,
#     show_progress=True,
# )

# from llama_index.core import StorageContext
# storage_context = StorageContext.from_defaults(graph_store=graph_store)

# index = KnowledgeGraphIndex.from_documents(
#     documents,
#     embed_model=embed_model,
#     storage_context=storage_context,
#     show_progress=True,
# )

# Create an index from the existing knowledge graph
index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    embed_model=embed_model,
)

In [5]:
# Define retriever
retriever = index.as_retriever(include_text=False)
# results = retriever.retrieve("Concentration et Densité Commerciale")
results = retriever.retrieve("Quels sont les effectifs totaux du commerce de détail dans les couronnes périurbaines ?")
for record in results:
    print(record.text)

Champ -> Retenu -> Commerce de détail
Commerce de détail -> Employant -> Salarié du commerce
Commerce de détail -> Localisés dans -> Villes-centres
Sections principales -> Présentent -> Effectifs totaux
Tableau -> Montre -> Effectifs totaux
Data -> Present -> Effectifs totaux
Modèle développement appareil commercial -> S'étend -> Couronnes périurbaines
Couronnes périurbaines -> Représentent -> 7% à 8% de l'appareil commercial national
Couronnes périurbaines -> Densité commerciale -> Croissante
Couronnes périurbaines -> Se situent -> Pôles de taille moyenne la plus faible
Commerce -> Croissance -> Couronnes périurbaines
Pôles secondaires -> Nombreux -> Couronnes périurbaines
Densité -> Accrue -> Couronnes périurbaines
Zones -> Être -> Couronnes périurbaines
Commerce -> Atteint ses plus forts degrés de spécialisation -> Couronnes périurbaines
Zones -> Occuper -> Couronnes périurbaines
Pôles commerciaux secondaires -> Localisés -> Couronnes périurbaines
Aire urbaine paris -> Concentre -> 

In [6]:
query_engine = index.as_query_engine(include_text=True)
# response = query_engine.query("comment a évolué la densité commerciale? Réponds en français.")
response = query_engine.query("Quels sont les effectifs totaux du commerce de détail dans les couronnes périurbaines ?")
print(response)

Les effectifs totaux du commerce de détail dans les couronnes périurbaines étaient de 112 703 en 2002.
